In [ ]:
def f(x,lmbda):
    return 0.5 * lmbda * np.dot(x, x) + 0.5 * np.linalg.norm(np.dot(A, x) - y)**2

def grad(x,lmbda):
    return lmbda * x + np.dot(A.T, np.dot(A, x) - y)

def hessian(lmbda):

    return lmbda * np.eye(A.shape[1]) + np.dot(A.T, A)



def dk_f(x0,lmbda):
  return np.linalg.inv(hessian(lmbda))

def get_alpha(x, alpha0, rho, gamma,lmbda):
  Dk = dk_f(x,lmbda)
  alpha = alpha0
  pk = -grad(x,lmbda)
  while f(x + alpha*Dk@pk,lmbda) > (f(x,lmbda) + gamma*alpha*grad(x,lmbda)@Dk@pk):
    alpha = rho*alpha
  return alpha

def newton_method_with_backtracking(x0,tol,alpha0, rho, gamma,lmbda):
  x = np.copy(x0)
  count = 0
  pk = grad(x,lmbda)
  xs = []
  xs.append(x)

  while np.linalg.norm(pk)>tol:
    Dk = dk_f(x,lmbda)
    alpha = get_alpha(x, alpha0, rho, gamma,lmbda)
    x = x - alpha * Dk @ pk
    pk = grad(x,lmbda)
    xs.append(x)
    count += 1



  return count, x, f(x,lmbda), xs

In [ ]:
lmbda = 1e-3
alpha0 = 0.99
rho = 0.5
gamma = 0.5
tol = 1e-4

#Newton

In [ ]:
#For each value of dimension in the ds array , we will check the behavior of Newton method

for i in range (np.size(ds)) :
  d = ds[i]
  A = np.random.randn (N,d)
# Normalize the columns
  for j in range ( A . shape [1]) :
    A [:,j] = A[: , j ]/ np.linalg.norm(A[: , j ])
  xorig = np.ones(( d ,1) )
  y = np.dot (A ,xorig ) + eps
  y = y.flatten()
  start = timeit . default_timer ()

  newtontime = timeit.default_timer () - start
  x0 = np.zeros(d)
  iterations,minimizer,final_value,xks = newton_method_with_backtracking(x0,tol,alpha0,rho, gamma,lmbda)
  x_opt = final_value
  newtontime = timeit.default_timer() - start
  print('--------------------------------------------------------------------')
  print('Dimension = ',d)
  print('Iteration = ',iterations )
  print('Time Taken = ',newtontime)
  print('||A*x_star - y||^2 = ',np.linalg.norm(A@minimizer - y)**2)
  print('|| x_opt - xorig ||^2 = ',np.linalg.norm(minimizer - xorig.flatten())**2 )



--------------------------------------------------------------------
Dimension =  1000
Iteration =  3
Time Taken =  0.614938902999711
||A*x_star - y||^2 =  7.581930383688819e-05
|| x_opt - xorig ||^2 =  826.824326366857
--------------------------------------------------------------------
Dimension =  5000
Iteration =  4
Time Taken =  65.81816025399985
||A*x_star - y||^2 =  8.892594751209928e-06
|| x_opt - xorig ||^2 =  4806.01343113296
--------------------------------------------------------------------
Dimension =  10000
Iteration =  4
Time Taken =  496.7143852680001
||A*x_star - y||^2 =  4.733275181840071e-06
|| x_opt - xorig ||^2 =  9780.120415921576


**Here it is seen that, for the newtons method, it is successful upto d = 10000**

#BFGS

In [ ]:
def get_alpha_bfgs(x, alpha0, rho, gamma, Bk,lmbda):
  alpha = alpha0
  pk = -grad(x,lmbda)
  while f(x + alpha*Bk@pk,lmbda) > (f(x,lmbda) + gamma*alpha*grad(x,lmbda)@Bk@pk):
    alpha = rho*alpha
  return alpha

def bfgs(x0, tol, alpha0, rho, gamma,lmbda,max_iter=500):

  x = np.copy(x0)
  n = len(x0)
  Bk = np.eye(n)
  count = 0
  pk = grad(x,lmbda)
  xs = []
  xs.append(x)
  while (np.linalg.norm(pk)>tol):
    if count > max_iter:
      break

    alpha = get_alpha_bfgs(x, alpha0, rho, gamma, Bk,lmbda)
    xnext = x - alpha*(Bk@pk)
    sk = xnext - x
    yk = grad(xnext,lmbda) - grad(x,lmbda)
    # BFGS
    Bk = np.dot((np.eye(len(x)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(x)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

    x = xnext
    pk = grad(x,lmbda)
    xs.append(x)
    count += 1


  return count, x, f(x,lmbda), xs


In [ ]:
import numpy as np
import timeit
lmbda = 1e-3
alpha0 = 0.99
rho = 0.5
gamma = 0.5
tol = 1e-5

In [ ]:
# Code for BFGS method
import numpy as np
import timeit
np . random . seed (1000) #for repeatability
N = 200
ds = [1000 , 5000 , 10000 , 20000 , 25000 , 50000 , 100000 , 200000 , 500000 , 1000000]
lambda_reg = 0.001
eps = np . random . randn (N ,1) # random noise
#For each value of dimension in the ds array , we will check the behavior of BFGS method
for i in range(np.size(ds)):
    d = ds[ i ]
    A = np.random.randn(N,d) # Normalize the columns
    for j in range(A.shape[1]) :
        A [: , j ] = A [: , j ]/ np . linalg . norm ( A [: , j ])
    xorig = np.ones(( d ,1) )
    y = np.dot(A,xorig) + eps
    y = y.flatten()
    start = timeit.default_timer ()
    x0 = np.zeros(d)
    iterations2,minimizer2,final_value2,xks2 = bfgs(x0, tol, alpha0, rho, gamma,lmbda, max_iter=500)
    bfgstime = timeit . default_timer () - start # time is in seconds
    print('--------------------------------------------------------------------')
    print('Dimension = ',d)
    print('Iteration = ',iterations2 )
    print('Time Taken = ',bfgstime)
    print('||A*x_star - y||^2 = ',np.linalg.norm(A@minimizer2 - y)**2)
    print('|| x_opt - xorig ||^2 = ',np.linalg.norm(minimizer2 - xorig.flatten())**2 )




--------------------------------------------------------------------
Dimension =  1000
Iteration =  33
Time Taken =  7.278909547000012
||A*x_star - y||^2 =  5.6726434545112945e-05
|| x_opt - xorig ||^2 =  865.3153032028484
--------------------------------------------------------------------
Dimension =  5000
Iteration =  46
Time Taken =  861.6070387549998
||A*x_star - y||^2 =  9.793228311327452e-06
|| x_opt - xorig ||^2 =  4783.681693038774
--------------------------------------------------------------------
Dimension =  10000
Iteration =  48
Time Taken =  6650.432980889
||A*x_star - y||^2 =  3.568022008658556e-06
|| x_opt - xorig ||^2 =  9829.015751366156


**Here it is seen that, for the BFGS method, it is successful upto d = 10000**